In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

from sklearn.externals import joblib
import holoviews as hv
from lib.models import plot_lrf
hv.extension('bokeh')

# MCA

In [ ]:
mca = joblib.load("../data/ml/ngaqua/mca.pkl")

In [ ]:
mca_mod = mca['model'].named_steps['mca']

In [ ]:
mca_mod.x_components_.shape

In [ ]:
hv.HoloMap({k: hv.Curve(c) for k, c in enumerate(mca_mod.x_components_.T)})

# Visualizing lrfs

In [ ]:
def get_lrf(lm_data):
    lrf, in_idx, out_idx = lm_data['mat'], lm_data['features']['in'], lm_data['features']['out']
    return pd.DataFrame(lrf, index=in_idx, columns=out_idx)

In [ ]:
w = np.asarray(xr.open_dataarray("../data/processed/ngaqua/w.nc"))


lm_data = joblib.load("../data/ml/ngaqua/linear_model.pkl")
lrf = get_lrf(lm_data)

p = xr.open_dataset("../data/ngaqua/stat.nc").p
plot_lrf(lrf, p, input_vars=['QT', 'SL', 'LHF', 'SHF'],
         output_vars=['Q1c', 'Q2']);

In [ ]:
def myquad(key, **kwargs):    
    m,n = lrf_pane.shape
    return quadmesh((p[:n], p[:m], lrf_pane), **kwargs)


def make_row(output_var, lrf, p):
    row = myquad(lrf[(output_var, 'qt')], p, label="QT", group=output_var) + \
          myquad(lrf[(output_var, 'sl')], p, label="SL",  group=output_var) + \
          hv.Curve(lrf[(output_var, 'shf')][0,:], label="SHF",  group=output_var) * \
          hv.Curve(lrf[(output_var, 'lhf')][0,:], label="LHF",  group=output_var)
    
    return row


opts = {'Curve': {'plot': dict(invert_axes=True, width=200),
                 'norm': dict(axiswise=True)},
        'QuadMesh':{'plot': dict(colorbar=True, invert_yaxis=True, invert_xaxis=True, invert_axes=True),
                   'style': dict(cmap='viridis'),
                   'norm': dict(axiswise=True)}}


def plot_lrf(lrf, p):
    return (make_row('Q1c', lrf, p) + make_row('Q2', lrf, p)).cols(3).opts(opts)
    


plot_lrf(lrf, p)

## First weighting the LRF by the projection operator

This is achieved by $\phi^{\dagger} \phi M$.  Where $\phi$ has shape (n_components, n_features)

In [ ]:
data = joblib.load("../data/ml/ngaqua/data.pkl")
mca_data = joblib.load("../data/ml/ngaqua/mca.pkl")
lm_data = joblib.load("../data/ml/ngaqua/linear_model.pkl")

lrf = get_lrf(lm_data)

feats = lm_data['features']['in']
mca = pd.DataFrame(mca_data['projection'], index=feats, columns=feats)

In [ ]:
plot_lrf(lrf, p, input_vars=)

In [ ]:
import pandas as pd

In [ ]:
pd.

In [ ]:
projection = mca_data['projection']

This means that each pane as size (nfeats, nout)

In [ ]:
output_vars = ['Q1c', 'Q2']
input_vars = ['qt', 'sl', 'shf', 'lhf']


def pad_to_match(lrf, mca, input_vars, output_vars):
    """Pad lrf to match mca"""

    lrf_padded = {}

    for in_var in input_vars:
        n_mca = mca[in_var].shape[0]
        n_lrf = lrf[('Q1c', in_var)].shape[0]

        for out_var in output_vars:
            key = (out_var, in_var)
            padded_pane = np.pad(lrf[key], ((0, n_mca-n_lrf), (0,0)), 'constant')
            lrf_padded[key] = padded_pane
            
    return lrf_padded

In [ ]:
lrf_padded = pad_to_match(lrf, mca_data['mat'],
                         input_vars, output_vars)


phi_dict = mca_data['mat']
phi = np.hstack(phi_dict[key].T for key in input_vars)

M  = np.vstack(np.hstack(lrf_padded[(out_var, in_var)].T for in_var in input_vars)
                    for out_var in output_vars).T

M_filt = projection @ M

Let's visualize this projection operator.

In [ ]:
hv.Raster(projection[:-2,:-2])

In [ ]:
%%opts Raster[invert_axes=True, invert_yaxis=True colorbar=True](cmap='viridis')
%%opts Curve [invert_axes=True width=150] {+axiswise}
hv.Raster(M_filt[:-2,:])  + hv.Curve(M_filt[-2,:]) * hv.Curve(M_filt[-1,:])

When using the projection operator associated with only two modes, the linear response functions are somewhat cleaned up and show some gross features of the LRF. Namely, QT (left half) tends to promote convection whereas SL (right half) supresses convection.